In [ ]:
import skimage
from skimage.transform import resize
import numpy as np
import ct_utils 
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display

def todo():
    raise NotImplementedError("In dieser Zelle gibt es noch mindestens ein TODO!")

# CT-Bildgebung
Die folgenden Zellen visualisieren, wie Daten in Computertomographen gemessen werden.

In [ ]:
# Definiere eine ground truth (gt) - ein "Phantom"-Bild, siehe [https://de.wikipedia.org/wiki/Shepp-Logan-Phantom] (ctrl/cmd + click)
x_gt = skimage.img_as_float(skimage.data.shepp_logan_phantom())

# Definiere den Vorwärtsoperator - bei CT ist das die Radontransformation
theta = np.linspace(0,180, endpoint = False, num=400)
A = ct_utils.Radon(theta)

# Clean data
y_clean =  A(x_gt)

fig, ax = plt.subplots(1,2,figsize=(10,5))
ax[0].imshow(x_gt, cmap='gray')
ax[0].axis('off')
ax[0].set_title(r'Ground Truth $x$')

ax[1].imshow(y_clean, cmap='gray')
ax[1].axis('off')
ax[1].set_title(r'Clean data $y_{\text{clean}} = Ax$');

In [ ]:
# Visualize how the sinogram is created by rotation of specimen/detector
ray_pos = np.arange(20, 400, step=40)

def plot_at_angle(angle):
    deg = 180/400*angle
    _, axs = plt.subplots(1, 3, figsize=(20,5))
    rot = skimage.transform.rotate(x_gt, deg)
    rot[:,ray_pos] = 0.5
    rot[:,ray_pos+1] = 1
    rot[:,ray_pos+2] = 0.5

    # plot the rotated phantom
    axs[0].imshow(rot, cmap = 'gray')
    axs[0].tick_params(bottom = False, left = False)
    axs[0].set_xticks([])
    axs[0].set_yticks([])
    axs[0].set_title('Source')
    axs[0].set_xlabel('Detector')

    # Plot the data slice at the current angle
    axs[1].plot(np.arange(400), y_clean[:,angle])
    axs[1].set_ylim([0,0.4])
    axs[1].set_title('Measurement at '+str(deg)+'$^{\circ}$')
    axs[1].set_xlabel('position')
    axs[1].set_ylabel('intensity decay')

    # Plot the whole acquired data up to current angle
    axs[2].imshow(y_clean[:,:angle+1], cmap = 'gray')
    axs[2].set_xlim([0,400])
    axs[2].set_xticks([0,100,200,300,400], ['$0^{\circ}$', '$45^{\circ}$', '$90^{\circ}$', '$135^{\circ}$', '$180^{\circ}$'])
    axs[2].set_xlabel('angle')
    axs[2].set_ylabel('position')

slider = widgets.IntSlider(min = 0, max = 399, step = 10, value = 0, continuous_update = True)
interactive_plot = interactive(plot_at_angle, angle = slider)
display(interactive_plot)

# Rekonstruktion aus unvollständigen Daten

Am Differentiations-Beispiel haben wir bereits gesehen, dass Regularisierung notwendig ist, um Instabilitäten/Fehleramplikation entgegenzuwirken. Die anderen beiden Kriterien von Gutgestelltheit sind aus theoretischer Sicht leichter zu garantieren, in der Praxis bereitet aber z.B. auch die Unterbestimmtheit Probleme.
Die folgende Zelle zeigt, dass selbst eine Rekonstruktion aus fehlerfreien, aber unvollständigen Daten im CT problematisch sein kann.

In [ ]:
def plot_lact(idx):
    angle = 399-idx
    deg = 180/400*angle
    lact_sino = y_clean[:,:angle+1]
    R_lact = ct_utils.Radon(theta[:angle+1])
    reco = R_lact.inv(lact_sino)
    _, axs = plt.subplots(1, 2, figsize=(10,5))

    # Incomplete Data plot
    axs[0].imshow(lact_sino, cmap='gray')
    axs[0].set_xlim([0,400])
    axs[0].set_xticks([0,100,200,300,400], ['$0^{\circ}$', '$45^{\circ}$', '$90^{\circ}$', '$135^{\circ}$', '$180^{\circ}$'])
    axs[0].set_xlabel('angle')
    axs[0].set_ylabel('position')
    axs[0].set_title('Sinogram of a scan of '+str(deg)+ ' degrees')

    # Image reconstruction plot
    axs[1].imshow(reco, cmap='gray')
    axs[1].axis('off')
    axs[1].set_title('Reconstruction')

slider2 = widgets.IntSlider(min = 0, max = 399, step = 30, value = 0, continuous_update = True)
interactive_plot2 = interactive(plot_lact, idx = slider2)
display(interactive_plot2)

# Fehler von naiver CT-Rekonstruktion in verschiedenen Dimensionen

In [ ]:
dims = [200, 300, 400, 500, 600, 700, 800]

# Für die Visualisierung speichern wir die Rekonstruktionen und Fehler bei verschiedenen Auflösungen
y_noisy_store = [] # liste für noisy Sinogramme
x_rec_store = [] # Liste für Rekonstruktionen
err_rec_store = np.zeros(len(dims)) # Liste für Rekonstruktionsfehler

for i, dim in enumerate(dims): # Iteration über verschiedene Problemdimensionen / Bildauflösungen
    x_resized = todo() # Verändere hier die Größe der ground truth zu (dim, dim)

    theta_resized = np.linspace(0,180, endpoint = False, num=dim)
    R_res = ct_utils.Radon(theta_resized)
    
    epsilon = todo()
    y_noisy = todo()
    x_rec = todo() # Hinweis: ct_utils.Radon hat eine Methode .inv zum Anwenden einer (bereits leicht regularisierenden) Inversen

    # store results
    y_noisy_store.append(y_noisy)
    x_rec_store.append(x_rec)
    err_rec_store[i] = np.linalg.norm(x_rec - x_resized)/dim


In [ ]:
def plot_resolutions(idx):
    _, axs = plt.subplots(1, 3, figsize=(20,5))

    axs[0].imshow(y_noisy_store[idx], cmap='gray', vmin=0, vmax=0.5)
    axs[0].axis('off')
    axs[0].set_title('Noisy Sinogram')

    axs[1].imshow(x_rec_store[idx], cmap='gray', vmin=0, vmax=1)
    axs[1].axis('off')
    axs[1].set_title('Reconstructed Phantom')

    axs[2].plot(dims[:idx+1], err_rec_store[:idx+1], alpha=0.9)
    axs[2].scatter(dims[idx], err_rec_store[idx], label='CT')
    axs[2].set_title('Per-Pixel Error')
    axs[2].set_xlabel('Dimension')
    axs[2].set_xlim([180,820])
    axs[2].set_xticks(dims)
    axs[2].set_ylim([0,1])
    axs[2].legend(loc='upper left')

    plt.suptitle('Resolution '+str(dims[idx]) + r'$\times$' + str(dims[idx]))


slider3 = widgets.IntSlider(min = 0, max = len(dims)-1, step = 1, value = 0, continuous_update = True)
interactive_plot3 = interactive(plot_resolutions, idx = slider3)
display(interactive_plot3)